# Data modifier

This notebook helps to convert USDA database files into a JSON format. The database can be downloaded from: https://ndb.nal.usda.gov/ndb/

Note that the script is designed to work with ASCII files (for both SR Legacy and BFPD files).

In [1]:
import pandas as pd
import numpy as np

In [2]:
def remove_trailing(val_str):
    return val_str.split('^')[0].rstrip('.')
def remove_leading(val_str):
    return val_str.split('~')[1]
def prefer_raw(val_str):
    if 'raw' in val_str.lower():
        return 1
    else:
        return 0
def prefer_zero(val_str):
    return 0
def simplifyName(val_str):
    split_name = val_str.split(',')
    return split_name[0].strip()
def lowerCase(val_str):
    return val_str.lower();
def modify_nut_value(x):
    x1 = x['dockey']
    x2 = x['nutrition_value']
    if(x1 in product_data_sr.index):
        if(product_data_sr.loc[x1,'weight'] == np.NaN):
            return x2
        else:
            return x2/100.0 * product_data_sr.loc[x1,'weight']
    else:
        return x['nutrition_value']
def remove_duplicates(groupDF):
    if(groupDF['dockey'].size > 1):
        groupDF = groupDF.drop(groupDF.loc[groupDF['serving_size_UOM'] == "oz"].index).copy()
        returnDF = groupDF.drop_duplicates(subset="dockey")
        return returnDF
    return groupDF

### Path Configuration/Reading Data

In [3]:
# SR Legacy Paths
food_des_sr_path = '/Users/Shanth/Documents/instaFresh_DATA/USDA_data/LEGACY/FOOD_DES.txt'
nut_data_sr_path = '/Users/Shanth/Documents/instaFresh_DATA/USDA_data/LEGACY/NUT_DATA.txt'
serving_size_sr_path = '/Users/Shanth/Documents/instaFresh_DATA/USDA_data/LEGACY/WEIGHT.txt'

# BFPD Paths
product_path = '/Users/Shanth/Documents/instaFresh_DATA/USDA_data/BFPD/Products.csv'
nutr_path = '/Users/Shanth/Documents/instaFresh_DATA/USDA_data/BFPD/Nutrients.csv'
serving_size_path = '/Users/Shanth/Documents/instaFresh_DATA/USDA_data/BFPD/Serving_size.csv'

# Output Paths
product_out_path_sr = '/Users/Shanth/Documents/instaFresh_DATA/JSON_data/product_data_sr_simplified.json'
nut_out_path_sr = '/Users/Shanth/Documents/instaFresh_DATA/JSON_data/nut_data_sr.json'

product_out_path_bfpd = '/Users/Shanth/Documents/instaFresh_DATA/JSON_data/product_data.json'
nut_out_path_bfpd = '/Users/Shanth/Documents/instaFresh_DATA/JSON_data/nut_data.json'

# Read BFPD data
my_cols = [str(i) for i in range(8)]
product_data = pd.read_csv(product_path)
nutrients_data = pd.read_csv(nutr_path)
serving_data = pd.read_csv(serving_size_path)

# Read SR Legacy data
product_data_sr = pd.read_csv(food_des_sr_path, sep="\~\^\~", header=None, names=my_cols)[['0','1','2']].copy()
nut_data_sr = pd.read_csv(nut_data_sr_path, sep="\~\^", header=None, names=my_cols)[['0','1','2']].copy()
serving_data_sr = pd.read_csv(serving_size_sr_path, sep="\~\^\~|\~\^|\^\~", header=None, names=['dockey','seq','serving_size','serving_size_UOM','weight'])[['dockey','serving_size','serving_size_UOM','weight']].copy()

/Users/Shanth/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/Shanth/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:25: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
/Users/Shanth/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:26: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
/Users/Shanth/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:27: ParserWarning: Falling back to t

### Data Cleanup

In [4]:
# Clean up serving size SR data

# clean tildas and do data type corrections
serving_data_sr['dockey'] = serving_data_sr['dockey'].apply(remove_leading)
serving_data_sr['weight'] = serving_data_sr['weight'].apply(remove_trailing)
serving_data_sr['weight'] = serving_data_sr['weight'].astype(float)

# group based on id and iterate
serving_data_sr_groups = serving_data_sr.groupby(['dockey']);

# remove duplicates
serving_data_sr = serving_data_sr_groups.apply(remove_duplicates)
serving_data_sr.reset_index(inplace=True, drop=True, level=1)
serving_data_sr = serving_data_sr[['serving_size', 'serving_size_UOM', 'weight']].copy()

In [5]:
# Cleanup SR product data
product_data_sr['0'] = product_data_sr['0'].apply(remove_leading)
product_data_sr['preference'] = product_data_sr['2'].apply(prefer_raw)
product_data_sr = product_data_sr[['0','2','preference']].copy()
product_data_sr.columns = ['dockey', 'name', 'preference']
product_data_sr = product_data_sr.set_index('dockey')

# Merge with serving data
product_data_sr = product_data_sr.join(serving_data_sr)

# Simplify name and remove duplicates
# Note, comment the two lines below if you dont want to edit the product data
product_data_sr['name'] = product_data_sr['name'].apply(simplifyName)
product_data_sr.drop_duplicates(subset='name',inplace=True)

# change nan values
product_data_sr.loc[product_data_sr['serving_size_UOM'].isnull(),'serving_size'] = 100
product_data_sr.loc[product_data_sr['serving_size_UOM'].isnull(),'serving_size_UOM'] = 'g'

# Create a lower case column
product_data_sr['lower_case_name'] = product_data_sr['name'].apply(lowerCase)


In [6]:
# Cleanup SR nutrient data
nut_data_sr['2'] = nut_data_sr['2'].apply(remove_trailing)
nut_data_sr['1'] = nut_data_sr['1'].apply(remove_leading)
nut_data_sr['0'] = nut_data_sr['0'].apply(remove_leading)
nut_data_sr.columns = ['dockey','nutrition_code','nutrition_value']
nut_data_sr['nutrition_code'] = pd.to_numeric(nut_data_sr['nutrition_code'])
nut_data_sr['nutrition_value'] = pd.to_numeric(nut_data_sr['nutrition_value'])
nut_data_sr['dockey'] = nut_data_sr['dockey'].astype(str)
nut_data_sr['nutrition_value'] = nut_data_sr.apply(modify_nut_value, axis=1)

# Delete weight column from product_data_sr
product_data_sr = product_data_sr.drop('weight',axis=1)

In [7]:
# Cleanup product data
product_data['preference'] = product_data['long_name'].apply(prefer_zero)
product_data = product_data[['NDB_Number', 'long_name', 'gtin_upc', 'preference']].copy()
product_data['NDB_Number'] = product_data['NDB_Number'].astype(str)
product_data.columns = ['dockey','name','upc','preference']
product_data = product_data.set_index('dockey')
serving_data = serving_data.set_index('NDB_No')
serving_data.index = serving_data.index.astype(str)
product_data = product_data.join(serving_data)
product_data = product_data[['name', 'upc', 'preference', 'Household_Serving_Size', 'Household_Serving_Size_UOM']].copy()
product_data.columns = ['name','upc','preference', 'servingSize', 'servingSizeUOM']

In [8]:
# Cleanup nutrient data
nutrients_data = nutrients_data[['NDB_No', 'Nutrient_Code', 'Output_value']]
nutrients_data.columns = ['dockey','nutrition_code','nutrition_value']
nutrients_data['dockey'] = nutrients_data['dockey'].astype(str)

In [9]:
# Dump nut and product SR data
nut_data_sr.to_json(nut_out_path_sr, orient='index')
product_data_sr.to_json(product_out_path_sr, orient='index')

In [10]:
# Dump nut and product data
nutrients_data.to_json(nut_out_path_bfpd, orient='index')
product_data.to_json(product_out_path_bfpd, orient='index')

In [14]:
nut_data_sr

dockey  nutrition_code  nutrition_value
0       01001             208         35.85000
1       01001             262          0.00000
2       01001             263          0.00000
3       01001             268        149.95000
4       01001             301          1.20000
5       01001             304          0.10000
6       01001             305          1.20000
7       01001             306          1.20000
8       01001             307         32.15000
9       01001             318        124.95000
10      01001             319         33.55000
11      01001             320         34.20000
12      01001             321          7.90000
13      01001             322          0.00000
14      01001             324          0.00000
15      01001             334          0.00000
16      01001             337          0.00000
17      01001             338          0.00000
18      01001             417          0.15000
19      01001             431          0.00000
20      01001             432          0.15000
21      01001             435          0.15000
22      01001             601         10.75000
23      01001             638          0.00000
24      01001             639          0.00000
25      01001             641          0.20000
26      01002             208        731.00000
27      01002             262          0.00000
28      01002             263          0.00000
29      01002             268       3058.00000
...       ...             ...              ...
644095  90560             631          0.09900
644096  90560             645          0.25900
644097  90560             646          0.25200
644098  93600             312          0.21250
644099  93600             404          0.10200
644100  93600             405          0.12750
644101  93600             406          0.93500
644102  93600             415          0.10200
644103  93600             606          0.10795
644104  93600             607          0.00000
644105  93600             608          0.00000
644106  93600             609          0.00000
644107  93600             610          0.00000
644108  93600             611          0.00000
644109  93600             612          0.00170
644110  93600             613          0.07650
644111  93600             614          0.02975
644112  93600             617          0.06205
644113  93600             618          0.02805
644114  93600             619          0.01445
644115  93600             620          0.04420
644116  93600             621          0.02805
644117  93600             626          0.01275
644118  93600             627          0.00000
644119  93600             628          0.00000
644120  93600             629          0.01955
644121  93600             630          0.00000
644122  93600             631          0.01020
644123  93600             645          0.07480
644124  93600             646          0.14450

[644125 rows x 3 columns]